# Merge mapping outputs from different classifications (wikidata, dictionaries, self & spacy) -  Classifying special street designations

In [1]:
#Load all needed libraries
import pandas as pd
import time
import warnings
from functools import reduce  

Add all data

In [2]:
#Streetnames
streetnames = pd.read_csv('../Datapreparation/streetnames.csv', encoding='UTF-8-SIG', sep=';')

#Results from spacy classification
spacy = pd.read_csv('../Datapreparation/spacy_out.csv', encoding='UTF-8-SIG', sep=';')

#Results from classification using wikidata
wiki_person = pd.read_csv('mapping wiki person_out.csv', encoding='UTF-8-SIG', sep=';')

#Results from classification using dictionaries & self-classifier
dictionary = pd.read_csv('out_dictionary.csv', encoding='UTF-8-SIG', sep=';')

Merge these datasets

In [7]:
mapping = reduce(lambda left, right:     # Merge three pandas DataFrames
                     pd.merge(left , right,
                              on = ["STR_ESID"],
                              how = "outer"),
                     [streetnames, spacy, dictionary])


# Add wiki_person

Rules to define final specific street designation

In [8]:
mapping.loc[mapping['DIC_Ortschaft'].notna() , 'classif'] = 'Ortschaft'
mapping.loc[mapping['DIC_Berufe'].notna() , 'classif'] = 'Beruf'
mapping.loc[mapping['SPACY_FR_ENT'] == 'PER', 'classif'] = 'Person'
mapping.loc[mapping['SPACY_DE_ENT'] == 'PER', 'classif'] = 'Person'
mapping.loc[mapping['DIC_Lebewesen'].notna() , 'classif'] = 'Lebewesen'
mapping.loc[mapping['DIC_Siedlung'].notna() , 'classif'] = 'Siedlung'
mapping.loc[mapping['DIC_Religion'].notna() , 'classif'] = 'Religion'
mapping.loc[mapping['DIC_Natur'].notna() , 'classif'] = 'Natur'
mapping.loc[mapping['DIC_Berg'].notna() , 'classif'] = 'Berg'
mapping.loc[mapping['DIC_Natur'].notna() , 'classif'] = 'Natur'
mapping.loc[mapping['DIC_Gewässer'].notna() , 'classif'] = 'Gewässer'
mapping.loc[mapping['STR_PREPS'] == 'sankt', 'classif'] = 'Religion'
mapping["classif"] = mapping["classif"].fillna('Andere')

Clean output dataset

In [9]:
mapping.columns

Index(['Unnamed: 0_x', 'STR_ESID', 'STN_LABEL_x', 'COM_FOSNR', 'COM_NAME',
       'COM_CANTON', 'STR_OFFICIAL', 'STR_EASTING', 'STR_NORTHING',
       'STN_LABEL_NO_BI', 'STN_LABEL_NO_TERMS', 'STR_TERMS', 'STR_PREPS',
       'STN_LABEL_FINAL_x', 'Unnamed: 0_y', 'STN_LABEL_y', 'STN_LABEL_FINAL_y',
       'SPACY_DE', 'SPACY_DE_LEMMA', 'SPACY_DE_ENT', 'SPACY_FR',
       'SPACY_FR_LEMMA', 'SPACY_FR_ENT', 'Unnamed: 0', 'STN_LABEL',
       'STN_LABEL_FINAL', 'DIC_Lebewesen', 'DIC_Ortschaft', 'DIC_Berufe',
       'DIC_Gewässer', 'DIC_Berg', 'DIC_Siedlung', 'DIC_Religion', 'DIC_Natur',
       'classif'],
      dtype='object')

In [10]:
mapping = mapping.drop(['Unnamed: 0_x', 'Unnamed: 0_y', 'STN_LABEL_y', 'STN_LABEL_FINAL_y','STN_LABEL_FINAL_x' ,'STN_LABEL_x'], axis=1)

In [11]:
mapping.to_csv('C:\CAS_Arbeit\cassda-zertifikatsarbeit\Modeling\out_mapping.csv', encoding='UTF-8-SIG', sep=';')